<a href="https://colab.research.google.com/github/iguv221/Tobigs_Assignment/blob/main/1%EC%A3%BC%EC%B0%A8%EA%B3%BC%EC%A0%9C_Framework_%EA%B9%80%EC%84%B1%EC%9A%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

### 필요한 클래스와 모듈들 임포트합니다.

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

### 더 빠른 딥러닝 학습을 위해 GPU 병렬 계산을 활성화 시킵니다.
### 없어도 CPU로 계산은 가능하지만 느립니다.

device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)
print(device + " is available")


cuda is available


In [2]:
### 딥러닝 학습하는데 parameter tuning 단계입니다.
### learning_rate 은 학습률로 딥러닝을 하는데 backpropagation 이 되는 정도를 결정합니다.
### 주로 Adam Optimizer 을 이용하는데 이때도 주로 권장되는 학습률은 0.0003 입니다다. 물론 얼마든지 변경 가능합니다.
### batch_size 은 mini-batch stochastic learning 을 하는데 사용하는 batch size 을 말합니다.
### batch_size 가 클수록 학습 시간은 늘어나지만 global optimized value 에 다가갈 확률이 늘어납니다.
### epochs 은 전체 데이터를 학습하는 횟수를 말합니다.

learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [3]:
### 데이터셋 불러오는 단계입니다.
### torchvision datasets 내의 MNIST 데이터셋을 불러오는데
### root 을 통해 저장 장소를 지정해주고 번거롭게 다시 다운 받는 과정을 생략할 수 있습니다.
### train 을 통해 학습 데이터셋 6만개를 받을지, 테스트 데이터셋 1만개를 받을지 결정할 수 있습니다.
### transform 을 통해 데이터셋을 필요한대로 변경할 수 있습니다.
### resize 을 이용해 모델에 쉽게 접목할 수 있게 변경하는 과정을 많이 거칩니다. 
### 그리고나서 ToTensor 을 이용해 GPU 에 사용할 수 있는 텐서로 변경해줍니다.
### 이 코드에는 ToTensor 만 이용했습니다.

train_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = True,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor() 
    ])
)

test_set = torchvision.datasets.MNIST(
    root = './data/MNIST',
    train = False,
    download = True,
    transform = transfroms.Compose([
        transfroms.ToTensor()
    ])
)

### 불러온 데이터를 DataLoader 에 로드하는 과정입니다.
### 이 과정을 거쳐야 개개의 data sample 들이 batch 으로 형성이 됩니다.
### 이 batch 을 통해 딥러닝 학습을 하기 때문에 꼭 거쳐야하는 과정입니다.
### 이때 shuffle 함수를 자주 이용해 랜덤성을 보장합니다.

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)


### 아래의 3줄은 딥러닝 모델을 학습하는데 직접적으로 필요한 과정은 아니고
### batch 의 dimension 을 확인하기 위해 있는 코드입니다.
### 이때 pytorch 의 dataloader 내 shape 은 (batch size, channel size, height, width) 순서대로 출력됩니다.
### tensorflow 의 dataloader 내 shape 은 (batch size, height, width, channel size) 순서대로 출력됩니다.

examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/MNIST/raw



torch.Size([1, 28, 28])

In [4]:
### Convolutional Network Class 을 정의하는 코드입니다.
### 우선 Class 에서 pytorch 상위 클래스 Module 을 상속합니다.
### 상속하는 이유는 간편하게 정해진 구도에 따라 모델을 설정하기 위함입니다.

### 아래에서 kernel_size 이란 convolutional neural network 을 실행하는데 kernel 의 크기를 말합니다.
### 5*5 의 frame 크기를 이용해 학습을 합니다. Conv2d 의 또 다른 parameter 으로 stride, padding, dilation 이 있습니다.

class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__() # super 을 이용해 overloading 을 시킵니다.
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) # conv1 함수를 선언하는 것으로, 채널수를 1 에서 10으로 늘립니다. 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) # conv2 함수를 선언하는 것으로, 채널수를 10 에서 20으로 늘립니다.
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) # drop2D 함수를 선언하는 것으로, 랜덤하게 노드 중 25% 을 제거합니다.
                                                          # 이를 하는 이유는 조금 더 빠른 학습시간과 과적합을 방지하기 위함입니다.
        self.mp = nn.MaxPool2d(2) # mp 함수를 선언하는 것으로, 2*2 frame 에서 최대값만을 남기는 과정입니다.
                                  # 이 과정을 거치는 이유는 유의미한 최대치인 weight 들을 많이 남기기 위함입니다.
        self.fc1 = nn.Linear(320,100) # fc1 함수를 선언하는 것으로, fully connected layer 을 만듭니다. 원래의 320 개 값을 100개로 줄입니다.
        self.fc2 = nn.Linear(100,10)  # fc2 함수를 선언하는 것으로, 위와 똑같습니다. 원래의 100 개 값을 10개로 줄입니다.
                                      # 10개로 줄인 이유는 MNIST 의 target label 들이 10개이기 때문입니다.


### 위의 init 모듈은 함수들을 선언하기만 하고 이용하지는 않았습니다.
### 그 과정은 아래의 forward 을 통해 실행이 됩니다.

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # 들어온 x 를 순서대로 conv1, mp, relu 시킵니다.
                                           # 원래의 loaded data 은 크기가 100, 1, 28, 28 인데 위의 과정을 거치면
                                           # 100, 10, 12, 12 가 됩니다. (channel 1->10 , (28-4)/2 = 12)
        x = F.relu(self.mp(self.conv2(x))) # 들어온 x 를 순서래도 conv2, mp, relu 시킵니다.
                                           # 원래의 loaded data 은 크기가 100, 10, 12, 12 인데 위의 과정을 거치면
                                           # 100, 20, 4, 4 가 됩니다. (channel 10>-20 , (12-4)/2 = 4)
        x = self.drop2D(x) # 들어온 x 를 dropout 25% 시킵니다. 이 과정에는 tensor size 는 변하지 않습니다.
        x = x.view(x.size(0), -1) # x 를 view 를 이용해 일직선으로 펼칩니다.
                                  # 100, 20, 4, 4 의 데이터가 하나의 point 은 20, 4, 4 이기 때문에 펼치면 320 이 됩니다. (20*4*4 = 320)
        x = self.fc1(x) # x 를 fully connected layer 인 fc1 에 통과시켜 320 을 100 으로 펼칩니다.
        x = self.fc2(x) # x 를 fully connected layer 인 fc2 에 통과시켜 100 을 10 으로 펼칩니다.
        return F.log_softmax(x) # 마지막으로 x 를 log_softmax 을 통과시켜 classification 중 가장 높은 확률을 최종 답으로 냅니다.

In [5]:
### model 을 선언하는 부분입니다.
### ConvNet() 을 통해 모델을 선언하고 to(device) 을 통해 GPU 로 넘깁니다.
### criterion 은 loss function 을 정의하는 부분으로 여기서는 CrossEntropyLoss 을 이용했습니다.
### Image Multi-class Classification 이 목적이기 때문에 CrossEntropyLoss 을 사용합니다.
### optimizer 함수를 통해 Adam optimizer 을 선언하고 model 의 parameter 들과 학습률 0.001 을 전달합니다.

model = ConvNet().to(device) 
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [6]:
### 본격적으로 모델을 학습시키는 부분입니다.
### for 문을 이용해 epoch 만큼 전체 데이터를 학습시킵니다.
### 이때 dataloader 은 train_data, train_target 순으로 저장이 되어 있기 때문에
### for data, target in train_loader 을 이용해 각자를 이용합니다.
### 이때도 GPU 활용을 위해 .to(device)으로 넘깁니다.

### optimizer 을 업로드 시키는데 보통 3줄의 코드가 사용됩니다.
### 그것은 각각 optimizer.zero_grad(), cost.backward(), 그리고 optimizer.step() 입니다.
### optimizer.zero_grad() 은 gradient 의 값들을 다시 초기화하는데 사용합니다.
### 이 과정이 필요한 이유는 이전의 gradient 값들을 초기화 하지 않은 채로 사용하면
### 그 값이 그대로 있는채로 업데이트가 되기 때문에 dependent gradients 가 되며 gradient explosion 으로 이어질 수 있습니다.
### 고로 안정적인 학습과 independent gradient 을 위해 사용합니다.
### loss.backward() 은 현재 loss 에 따라 backpropagation 하는데 필요한 미분값을 계산하고 현재의 parameter 에 업데이트 될 값을 설정합니다.
### optimizer.step() 은 예상값과 실제값의 차이를 반영하기 위해 backpropagation 을 진행하는 과정입니다.
### 이때 모델의 weight value 들이 수정이 됩니다.
### 사실 이 3줄은 대부분의 딥러닝 학습 과정에 그대로 구현이 되지만 자세히 알지 않아도 학습은 잘 됩니다...

### hypothesis = model(data) 을 통해 현 모델을 이용해 data 의 예상값을 추출합니다.
### cost = criterion(hypothesis, target) 을 이용해 CrossEntropyLoss 에 의거한 차이를 cost 에 저장을 합니다.
### 당연히 cost 가 작을수록 정확도가 높은 모델입니다.
### avg_cost += cost / len(train_loader) 은 모델 학습에 직접적으로 사용되는 코드는 아니지만
### batch 와 epoch 들을 거치면서 loss 가 잘 감소하고 있는지를 확인하기 위해 자주 사용됩니다.
### 그 밑의 print 을 통해 average loss 을 확인할 수 있으며 지속적으로 감소하는 값이 제일 이상적입니다.

for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() 
        # (주석 예) 모든 model의 gradient 값을 0으로 설정 - 초기화 이유 : ?
        hypothesis = model(data)
        cost = criterion(hypothesis, target) 
        cost.backward()
        optimizer.step() 
        avg_cost += cost / len(train_loader) 
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch:    1]  cost = 0.316545278
[Epoch:    2]  cost = 0.117382027
[Epoch:    3]  cost = 0.0888482556
[Epoch:    4]  cost = 0.07573203
[Epoch:    5]  cost = 0.0650398582


In [7]:
### 아래의 코드는 모델을 평가하는 부분입니다.
### 우선 model.eval() 을 통해 모델이 학습하는 단계가 아님을 선포합니다.
### 그리고 밑에 with torch.no_grad() 을 이용해 모델 내 weight 들이 더 이상 변하지 않게 바꿔줍니다.
### 이때 또다시 data 와 target 을 GPU 로 넘기는 과정을 거칩니다.
### 그리고 out = model(data) 은 위에서의 hypothesis = model(data) 와 똑같은 코드로 
### 완성된 모델에 현재 데이터를 삽입해 예측값을 추출합니다.
### 그렇게 예측값들을 추출하고 preds = torch.max(out.data,1)[1] 을 이용해 label 을 시키고
### 아래의 total 과 correct 을 이용해 전체 진행률과 정확도를 알 수 있습니다.
### 그리고 마지막에 print 을 이용해 모델이 얼마나 잘 학습했는지 알 수 있습니다.

model.eval()
with torch.no_grad(): 
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] 
        total += len(target) 
        correct += (preds==target).sum().item() 
        
    print('Test Accuracy: ', 100.*correct/total, '%')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Test Accuracy:  98.52 %
